In [3]:
import os
import json
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredFileLoader 
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache, set_debug
from langchain.storage import LocalFileStore
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.utils.math import cosine_similarity

In [4]:
# 언어 모델 초기화
print("🔧 Initializing language model...")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

🔧 Initializing language model...


In [4]:
# 프롬프트 템플릿 생성
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            """
            당신은 AI 어시스턴트입니다. 
            사용자가 질문에 대한 답변을 찾을 수 있도록 돕습니다.
            
            사용자의 질문 : {context}

            
            """
        ),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)


In [8]:
def rag_pipeline(question):
    # 입력된 질문에 대한 임베딩 생성
    print("🧠 Embedding the input question...")
    question_embedding = embeddings.embed_documents([question])[0]    
    # 기존 질문과 비교
    print("🔍 Comparing with existing documents...")
    results = retriever.get_relevant_documents(question)
    similarities = {}
    
    for doc in results:
        doc_content = doc.page_content.strip()
        if not doc_content:
            continue
        
        doc_embedding = embeddings.embed_documents([doc_content])[0]
        similarity = cosine_similarity([question_embedding], [doc_embedding])[0][0]
        similarity_percentage = similarity * 100
        similarities[doc_content] = similarity_percentage

    # 가장 유사한 질문 찾기
    most_similar_question = max(similarities, key=similarities.get, default=None)
    if most_similar_question:
        max_similarity = similarities[most_similar_question]
        

        if max_similarity > 80.0:  # 유사도가 충분히 높은 경우 관련 답변 사용
            context = f"질문이 '{most_similar_question}'와 매우 유사합니다."
            print(f"\n가장 유사한 질문: '{most_similar_question}'")
            print(f"유사도: {max_similarity:.2f}%")
            prompt = prompt_template.format(context=context, question=question)
            output = llm.invoke(prompt)
            return { "LLMAnswer": output.content, "userQuestion": question}
    
    return {"LLMAnswer": None, "userQuestion": question}

In [10]:
# 메인 함수
def main(user_input):
    response = rag_pipeline(user_input)
    print("Assistant:", response)

user_input = [
                "MAC에 대해서 자세히 알려주세요.",
                "컴파일러가 무엇인가요??", 
                "TCP와 UDP의 차이점은 무엇인가요?", 
                "데이터베이스에 대해 알려주세요.",
                "TCP의 이점은 무엇인가요?"
            ]

for i in user_input:
    main(i)
    print("\n-----------------------------------\n")
    


🧠 Embedding the input question...
🔍 Comparing with existing documents...
Assistant: {'LLMAnswer': None, 'userQuestion': 'MAC에 대해서 자세히 알려주세요.'}

-----------------------------------

🧠 Embedding the input question...
🔍 Comparing with existing documents...
Assistant: {'LLMAnswer': None, 'userQuestion': '컴파일러가 무엇인가요??'}

-----------------------------------

🧠 Embedding the input question...
🔍 Comparing with existing documents...

가장 유사한 질문: '5
2024-08-20
스위치와 허브의 차이점은?
스위치는 데이터 패킷을 특정 장치로 전달하는 반면, 허브는 모든 장치로 데이터를 전송합니다.
6
2024-08-20
IP 주소란?
IP 주소는 네트워크 상에서 장치들을 식별하기 위한 고유 주소입니다.
7
2024-08-20
DNS의 역할은?
DNS는 도메인 이름을 IP 주소로 변환하는 시스템입니다.
8
2024-08-20
방화벽이란?
방화벽은 네트워크 보안을 위해 외부의 불법적인 접근을 차단하는 시스템입니다.
9
2024-08-20
VPN의 기능은?
VPN은 공용 네트워크에서 사설 네트워크를 사용하는 것처럼 보안을 제공하는 기술입니다.
10
2024-08-20
패킷이란?'
유사도: 85.97%
Assistant: {'LLMAnswer': 'TCP(Transmission Control Protocol)와 UDP(User Datagram Protocol)는 둘 다 인터넷 프로토콜 스위트의 일부로, 데이터 전송을 위한 프로토콜입니다. 그러나 이 두 프로토콜은 데이터 전송 방식에서 여러 가지 중요한 차이점이 있습니다.\n\n1. **연결 지향

In [13]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 언어 모델 초기화
print("🔧 언어 모델 초기화 중...")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 프롬프트 템플릿 생성
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            """
            당신은 AI 어시스턴트입니다. 주어진 질문이 STT(음성 인식)로 변환된 텍스트라서 언어가 섞이거나 오타가 있을 수 있습니다.
            사용자가 입력한 질문을 정제하고, 정제된 질문을 바탕으로 명확한 답변을 제공하세요.
            
            교수가 답변한 원본 질문 : {original_question}
            
            교수가 답변한 원본 답변: {cleaned_question}
            
            교수가 답변한 원본 질문-답변에 기반하여 명확하고 간결한 답변을 제공하세요.
            """
        ),
        HumanMessagePromptTemplate.from_template("{cleaned_question}")
    ]
)

def clean_text(stt_text):
    # 간단한 텍스트 정제 - 영어/한글 섞임, 오타 수정 등
    cleaned_text = stt_text.replace(" ", "").replace("??", "?")  # 공백 제거 및 이중 물음표 수정 예시
    return cleaned_text

def rag_pipeline(stt_input):
    # STT 입력 텍스트 정제
    print("🧹 STT 입력 텍스트 정제 중...")
    cleaned_question = clean_text(stt_input)

    # LLM에 정제된 질문 전달
    print("💬 LLM을 통한 답변 생성 중...")
    prompt = prompt_template.format(original_question=stt_input, cleaned_question=cleaned_question)
    output = llm.invoke(prompt)
    
    return {"LLMAnswer": output.content, "userQuestion": cleaned_question}

🔧 언어 모델 초기화 중...


In [14]:
# 메인 함수
def main(stt_input):
    response = rag_pipeline(stt_input)
    print("Assistant:", response["LLMAnswer"])

# 테스트 입력
# 교수님 답변 STT을 client에서 POST로 줌 
# 해당 답변을 받아서 정제 후 LLM에 전달
# LLM의 답변을 반환

# 교수님의 말투를 반영한 답변 예시
responses = [
    "MAC에 대해서 말씀드리자면, 네트워크 상에서 각 장치가 서로를 구분할 수 있도록 하는 고유한 식별자입니다. 쉽게 말해서, 네트워크에 연결된 각 기기에 부여되는 일종의 '주소'라고 보시면 됩니다. 이 MAC 주소를 통해 데이터가 정확히 어떤 장치로 가야 하는지 결정됩니다.",
   
    "TCP와 UDP의 차이점에 대해 설명드리겠습니다. TCP는 연결 기반으로, 데이터 전송의 신뢰성을 보장합니다. 즉, 데이터가 정확하게 전달되도록 보장하는 것이죠. 반면에 UDP는 신뢰성보다는 속도에 중점을 둔 프로토콜입니다. 그래서 UDP는 실시간 전송이 중요한 경우, 예를 들어 게임이나 스트리밍 같은 상황에서 많이 사용됩니다.", 
    # STT 현실 반영 
    "TCP의 이점에 대해서 물어보셨는데요... 음... 그게 TCP는 이건ㄱ 안정적인 연결을 보장합니다. 데이터를 전송할 때, 잃어버리지 않고, 제대로 전송됐는지를 확인하기 때문에 중요한 정보나 파일을 전송할 때 유리합니다. 다만, 속도보다는 정확성을 중시한다는 점이 있고요... 네, 그래서 주로 이메일, 파일 전송 등에 사용됩니다."
]


for i in responses:
    main(i)
    print("\n-----------------------------------\n")

🧹 STT 입력 텍스트 정제 중...
💬 LLM을 통한 답변 생성 중...
Assistant: MAC 주소는 네트워크 상에서 각 장치를 구분하는 고유한 식별자입니다. 쉽게 말해, 네트워크에 연결된 각 기기에 부여되는 '주소'로, 이 MAC 주소를 통해 데이터가 정확히 어떤 장치로 전송되어야 하는지를 결정합니다.

-----------------------------------

🧹 STT 입력 텍스트 정제 중...
💬 LLM을 통한 답변 생성 중...
Assistant: TCP와 UDP의 차이점에 대해 설명하겠습니다.

TCP(Transmission Control Protocol)는 연결 기반의 프로토콜로, 데이터 전송의 신뢰성을 보장합니다. 즉, 데이터가 정확하게 전달되도록 확인하고, 손실된 데이터는 재전송합니다.

반면, UDP(User Datagram Protocol)는 신뢰성보다는 속도에 중점을 둔 프로토콜입니다. 데이터 전송의 확인 절차가 없기 때문에 빠른 전송이 가능하며, 주로 실시간 전송이 중요한 상황에서 사용됩니다. 예를 들어, 온라인 게임이나 스트리밍 서비스에서 많이 활용됩니다.

-----------------------------------

🧹 STT 입력 텍스트 정제 중...
💬 LLM을 통한 답변 생성 중...
Assistant: TCP(전송 제어 프로토콜)의 이점은 다음과 같습니다:

1. **안정적인 연결**: TCP는 데이터 전송 시 안정적인 연결을 보장합니다. 이는 데이터가 손실되지 않고 정확하게 전송되도록 확인하는 메커니즘을 갖추고 있기 때문입니다.

2. **신뢰성**: 데이터가 제대로 전송되었는지를 확인하고, 만약 데이터가 손실되면 재전송을 요청하는 기능이 있어 중요한 정보나 파일 전송에 유리합니다.

3. **정확성 중시**: TCP는 속도보다는 데이터의 정확성을 중시합니다. 이로 인해 중요한 데이터 전송에 적합합니다.

4. **주요 용도**: 주로 이메일, 파일 전송 등과 같은 애플리케이션에서 사용됩니다.

이러한 특

In [15]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 언어 모델 초기화
print("🔧 언어 모델 초기화 중...")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 프롬프트 템플릿 생성
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            """
            당신은 AI 어시스턴트입니다. 주어진 질문이 STT(음성 인식)로 변환된 텍스트라서 언어가 섞이거나 오타가 있을 수 있습니다.
            사용자가 입력한 질문을 정제하고, 정제된 질문을 바탕으로 명확한 답변을 제공하세요.
            
            교수가 답변한 원본 질문 : {original_question}
            
            교수가 답변한 원본 답변: {cleaned_question}
            
            교수가 답변한 원본 질문-답변에 기반하여 명확하고 간결한 답변을 제공하세요.
            """
        ),
        HumanMessagePromptTemplate.from_template("{cleaned_question}")
    ]
)

def clean_text(stt_text):
    # 간단한 텍스트 정제 - 영어/한글 섞임, 오타 수정 등
    cleaned_text = stt_text.replace(" ", "").replace("??", "?")  # 공백 제거 및 이중 물음표 수정 예시
    return cleaned_text

def rag_pipeline(stt_input):
    # STT 입력 텍스트 정제
    print("🧹 STT 입력 텍스트 정제 중...")
    cleaned_question = clean_text(stt_input)

    # LLM에 정제된 질문 전달
    print("💬 LLM을 통한 답변 생성 중...")
    prompt = prompt_template.format(original_question=stt_input, cleaned_question=cleaned_question)
    output = llm.invoke(prompt)
    
    return {"LLMAnswer": output.content, "userQuestion": cleaned_question}

# 메인 함수
def main(stt_input):
    response = rag_pipeline(stt_input)
    print("Assistant:", response["LLMAnswer"])

# 테스트 입력
# 교수님 답변 STT을 client에서 POST로 줌 
# 해당 답변을 받아서 정제 후 LLM에 전달
# LLM의 답변을 반환

# 교수님의 말투를 반영한 답변 예시
responses = [
    "MAC에 대해서 말씀드리자면, 네트워크 상에서 각 장치가 서로를 구분할 수 있도록 하는 고유한 식별자입니다. 쉽게 말해서, 네트워크에 연결된 각 기기에 부여되는 일종의 '주소'라고 보시면 됩니다. 이 MAC 주소를 통해 데이터가 정확히 어떤 장치로 가야 하는지 결정됩니다.",
   
    "TCP와 UDP의 차이점에 대해 설명드리겠습니다. TCP는 연결 기반으로, 데이터 전송의 신뢰성을 보장합니다. 즉, 데이터가 정확하게 전달되도록 보장하는 것이죠. 반면에 UDP는 신뢰성보다는 속도에 중점을 둔 프로토콜입니다. 그래서 UDP는 실시간 전송이 중요한 경우, 예를 들어 게임이나 스트리밍 같은 상황에서 많이 사용됩니다.", 
    # STT 현실 반영 
    "TCP의 이점에 대해서 물어보셨는데요... 음... 그게 TCP는 이건ㄱ 안정적인 연결을 보장합니다. 데이터를 전송할 때, 잃어버리지 않고, 제대로 전송됐는지를 확인하기 때문에 중요한 정보나 파일을 전송할 때 유리합니다. 다만, 속도보다는 정확성을 중시한다는 점이 있고요... 네, 그래서 주로 이메일, 파일 전송 등에 사용됩니다."
]


for i in responses:
    main(i)
    print("\n-----------------------------------\n")

🔧 언어 모델 초기화 중...
🧹 STT 입력 텍스트 정제 중...
💬 LLM을 통한 답변 생성 중...
Assistant: MAC 주소는 네트워크 상에서 각 장치를 구분하기 위한 고유한 식별자입니다. 쉽게 말해, 네트워크에 연결된 각 기기에 부여되는 '주소'로, 이 MAC 주소를 통해 데이터가 정확히 어떤 장치로 전송되어야 하는지를 결정합니다.

-----------------------------------

🧹 STT 입력 텍스트 정제 중...
💬 LLM을 통한 답변 생성 중...
Assistant: TCP와 UDP의 차이점에 대해 설명하겠습니다. 

TCP(Transmission Control Protocol)는 연결 기반의 프로토콜로, 데이터 전송의 신뢰성을 보장합니다. 즉, 데이터가 정확하게 전달되도록 확인하는 기능이 있습니다. 

반면, UDP(User Datagram Protocol)는 신뢰성보다는 속도에 중점을 둔 프로토콜입니다. 따라서 UDP는 실시간 전송이 중요한 경우, 예를 들어 게임이나 스트리밍과 같은 상황에서 많이 사용됩니다. 

요약하자면, TCP는 신뢰성을, UDP는 속도를 중시하는 프로토콜입니다.

-----------------------------------

🧹 STT 입력 텍스트 정제 중...
💬 LLM을 통한 답변 생성 중...
Assistant: TCP(전송 제어 프로토콜)의 이점은 다음과 같습니다:

1. **안정적인 연결 보장**: TCP는 데이터 전송 시 안정적인 연결을 제공합니다. 데이터가 손실되지 않고 정확하게 전송되었는지를 확인합니다.

2. **데이터 무결성**: 중요한 정보나 파일을 전송할 때, 데이터의 무결성을 보장하여 신뢰성을 높입니다.

3. **정확성 중시**: TCP는 속도보다는 데이터의 정확성을 더 중요하게 여깁니다. 이로 인해 전송 속도는 상대적으로 느릴 수 있지만, 데이터의 신뢰성이 확보됩니다.

4. **주요 용도**: 주로 이메일, 파일 전송 등에서 사용됩니다.

이러한 이유로 TCP는 